In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.integrate as intg

#References Zhang 2010
#W_ij    -> Eqn (11)
#Delta_i -> Eqn (12)
#f_ij    -> Eqn (17)

#constants
Omega0 = 0.2 #present day matter density in unit of critical density
OmegaLambda = 0.8
Omegak = 1-Omega0-OmegaLambda

deltaz = 0.2 #redshift bin width
z_limit = 4
z_res = 400 #in integration 1 redshift is broken into z_res intervals
gaussian_spread = 0.05
print('z resolution is %1.5f' %(1/z_res))
#for zlim = 4, zres = 400 ~20sec/run

#Redshift coordinates
#throw away the first element to perform right Riemann sum to avoid 0 division
z_list = np.linspace(0,z_limit,z_limit*z_res,dtype='float64')[1:]
z_i_list = np.linspace(z_lowerbound,z_upperbound,int(deltaz*z_res),dtype='float64')[1:]

################################################
#Setups - Calculating related functions
################################################

def get

#Gaussian Photo-Z PDF
photozPDF = np.empty((len(z_list),len(z_i_list)),dtype='float64')
for ztrue_index in range(len(z_list)):
    ztrue = z_list[ztrue_index]
    #std function as a linear in redshift
    #Zhang 2010 discusses when 0.05->0.03 accuracy improves by 2.3x on page 7 bottom right
    std = gaussian_spread*(1+ztrue)
    #gaussian pdf
    prefector = 1/(np.sqrt(2*np.pi)*std)
    photozPDF[ztrue_index] = prefector*np.exp(-0.5*((z_i_list-ztrue)/std)**2)
test_redshift_index=int(1.25*z_res)
plt.plot(z_i_list, photozPDF[test_redshift_index],label='area under the curve=%1.3f'%(sum(photozPDF[test_redshift_index]/z_res)))
plt.xlabel('redshift z')
plt.ylabel('photo_z PDF around z_true=%1.4f, \n with sigma_p=%1.3f' %(z_list[test_redshift_index],gaussian_spread))
plt.show()

#galaxy true z distribution
def n(z):
    return z*z*np.exp(-z/0.5)

#galaxy true z distribution normalized in bin i
n_i_norm = intg.quad(n,z_lowerbound,z_upperbound)[0]
def n_i(z):
    return z*z*np.exp(-z/0.5)/n_i_norm
n_i_list = n_i(z_i_list)

#galaxy true z distribution normalized from 0 to infinity=z_limit
n_all_norm = intg.quad(n,0,z_limit)[0]
def n_all(z):
    return z*z*np.exp(-z/0.5)/n_all_norm
n_all_list = n_all(z_list)

#galaxy photo z distribution (true z distribution + smear) normalized from 0 to infinity=z_limit
n_p_i_list = np.zeros(len(z_list))
z_i_list_index = 0
for ztrue in z_i_list:
    std = gaussian_spread*(1+ztrue)
    prefector = 1/(np.sqrt(2*np.pi)*std)
    n_p_i_list += n_i_list[z_i_list_index]*prefector*np.exp(-0.5*((z_list-ztrue)/std)**2)
    z_i_list_index += 1
n_p_i_list_norm = sum(n_p_i_list)/z_res
n_p_i_list = n_p_i_list/n_p_i_list_norm

#galaxy photo z distribution (true z distribution + smear) normalized in bin i
n_p_i_bin_list = np.zeros(len(z_i_list))
z_i_list_index = 0
for ztrue in z_i_list:
    std = gaussian_spread*(1+ztrue)
    prefector = 1/(np.sqrt(2*np.pi)*std)
    n_p_i_bin_list += n_i_list[z_i_list_index]*prefector*np.exp(-0.5*((z_i_list-ztrue)/std)**2)
    z_i_list_index += 1
n_p_i_bin_list_norm = sum(n_p_i_bin_list)/z_res
n_p_i_bin_list = n_p_i_bin_list/n_p_i_bin_list_norm

plt.plot(z_list, n_all_list,linestyle='dashed',label = 'n_all_list (truez) normalized from z=0 to z=inf=zlim')
plt.plot(z_list, n_p_i_list,label = 'n_p_i_list (photoz) normalized from z=0 to z=inf=zlim')
plt.plot(z_i_list, n_i_list,linestyle='dashed',label = 'n_z_i_list (truez) truncated and normalized in the %1i th redshift bin'%i)
plt.plot(z_i_list, n_p_i_bin_list,label = 'n_p_i_bin_list (photoz) truncated and normalized in the %1i th redshift bin'%i)
plt.xlabel('redshift z')
plt.ylabel('galaxy redshift distribution')
plt.legend(bbox_to_anchor=(1, 1.05))
plt.show()

#comoving distance function, in unit of c/H0
#Troxel and Ishak 2014 equation (8) without the factor of H0
chi_list = np.array([],dtype='float64')
def chi_integrand(z):
    return 1/np.sqrt(Omega0*(1+z)**3+Omegak*(1+z)**2+OmegaLambda)
def chi(z):
    return intg.quad(chi_integrand,0,z)[0]
for z in z_list: chi_list = np.append(chi_list,chi(z))
plt.plot(z_list, chi_list)
plt.xlabel('redshift z')
plt.ylabel('comoving distance chi(z) \n Omega0=%1.2f, OmegaLambda=%1.2f' %(Omega0,OmegaLambda))
plt.show()

#Lensing Kernel List
#W_L_list[z_list,z_list]
W_L_list = np.empty((len(z_list),len(z_list)),dtype='float64')
for z_G_index in range(len(z_list)):
    chi_G = chi_list[z_G_index]
    W_L_list[z_G_index] = 1.5*Omega0*(1+z_list)*chi_list*(1-chi_list/chi_G)
W_L_list[W_L_list<0]=0 #set negative entries to 0 (eqv. set chi_L>chi_G entries to 0)
    #test = W_L_list.clip(min=0) This is slightly slower than the above method
test_redshift_index = int(1.2*z_res)
plt.plot(z_list, W_L_list[test_redshift_index])
plt.xlabel('redshift z')
plt.ylabel('lensing kernel W_L with source z_G=%1.4f' %(z_list[test_redshift_index]))
plt.show()